
# Calculation of a length of a $p$ unit circle in the 2-dimensional plane $\mathbb{R}^2$ with the Minkowski distance of order $p$


Consider the following vector norm $\|\vec{v}\|_{p}$ = $\sqrt[p]{x^{\,p} + y^{\,p}}$ for $\vec{v}=\{x,y\}\in\mathbb{R}^2$ and $p>1$. Let's call a $p$ unit circle the set of points $\{(x,y) \in \mathbb{R}^{2}: \|\vec{v}\|_p=1\}$ and a self-length of the $p$ unit circle $-$ the length, calculated using the metric $d(\vec{u}, \vec{v}) = \|\vec{u} - \vec{v}\|_p$. The self-lengh of a $p$-unit circle will be denoted as $\mu_p$ or $\mu(p)$. <br><br>Suppose $\frac{1}{p} + \frac{1}{q} = 1$, where $p, q > 1$.  Then $\mu_p = \mu_q$.<br><br>Below is a table of the differences between the self-lengths of $p$- and $q$-unit circles, approximated by the section of a circle into $n$ sectors.


---
Суть наблюдения. Возьмем на плоскости (двумерном векторном пространстве) выпуклое симетричное тело T с центром в нуле, т.е. 

1. 0 ∈ T
1. ∀ v ∈ T: -v ∈ T 
1. ∀ u, v ∈ T, α ∈ (0,1): (1-α)v + αu ∈ T 
1. ∀u ∈ T ∃ α ∈ R: αu ∉ T 

Зададим этим телом норму векторов и соответственно метрику на плоскости. Посчитаем длинну единичной (в этой метрике) окружности. Проделаем то же в сопряженном пространстве. Полученные таким образом длины равны между собой. 

Если рассматривать пространства с метрикой Минковского, то выполняется равенство для собственных длин единичных окружностей в p- и q-метриках, где 1/p + 1/q = 1

In [1]:
import numpy as np
import pandas as pd

In [2]:
def p_norm_basic(vectors:np.ndarray, order:float) -> float:
    '''Return the Minkowski vector norm of order p for the vectors.
    
    vectors : np.ndarray of a shape (n,k), where n is a number of vectors,
        and k - a dimension of vectors.
    order : float > 1. An order of the Minkowski norm.
    '''

    vectors = np.atleast_2d(vectors)
    
    return np.sum(np.abs(vectors)**order, axis=1)**(1/order)

In [3]:
def p_norm_power(vectors:np.ndarray, order:float) -> float:
    '''Return the Minkowski vector norm of order p for the vectors.
    
    vectors : np.ndarray of a shape (n,k), where n is a number of vectors,
        and k - a dimension of vectors.
    order : float > 1. An order of the Minkowski norm.
    '''
    
    vectors = np.atleast_2d(vectors)
    
    return np.power(np.sum(np.power(np.abs(vectors), order), axis=1), 1/order)

In [4]:
def p_norm_linalg(vectors:np.ndarray, order:float) -> float:
    '''Return the Minkowski vector norm of order p for the vectors.
    
    vectors : np.ndarray of a shape (n,k), where n is a number of vectors,
        and k - a dimension of vectors.
    order : float > 1. An order of the Minkowski norm.
    '''

    vectors = np.atleast_2d(vectors)
    
    return np.linalg.norm(vectors, ord=order, axis=1)

In [5]:
def self_length(norm:callable, approximation:int=2) -> float:
    '''A length of a unit-circle {x: norm(x) == 1}, measured by norm.
    
    norm : A norm on a plane, symmetrical about the coordinate axes 
        and lines x = y, x = -y.
    approximation : int > 0, 2 by default. A number of sections 
        of the first 1/8 arc to approximate the length.'''

    arcs = 8 
    rotations = np.linspace(0, 2*np.pi/arcs, approximation+1)
    
    x_coordinates = np.cos(rotations)[:, np.newaxis]
    y_coordinates = np.sin(rotations)[:, np.newaxis]
    # improve precision with longdouble
    units = np.hstack([x_coordinates, y_coordinates]).astype(np.longdouble)
    # normalize units, so that norm(u) == 1 for each u in units
    coeff = np.apply_along_axis(lambda u: 1/norm(u), 1, units)
    units = units * coeff
    
    return arcs * np.sum(norm(units[1:] - units[:-1]))

In [6]:
from functools import partial
from time import monotonic

norm_collection = {
    'basic' : p_norm_basic, 
    'linalg': p_norm_linalg,
    'power' : p_norm_power
}

# P : initial norm orders
# Q : dual space norm orders
# N : approximations
P = np.arange(2, 500, 25) 
Q = P / (P-1) 
N = [1, 10, 100]

result = dict()

for norm_name, norm in norm_collection.items():
    
    p_norms = [partial(norm, order=p) for p in P]
    q_norms = [partial(norm, order=q) for q in Q]

    index = pd.MultiIndex.from_tuples(
        [(p, q, n) for p, q in zip(P,Q.round(4)) for n in N], 
        names=['p','q','approximation'])

    
    time_start = monotonic()
    # BEGIN main logic
    data_P = pd.Series([self_length(norm, n) for norm in p_norms for n in N], index)
    data_Q = pd.Series([self_length(norm, n) for norm in q_norms for n in N], index)
    delta = np.abs(data_P - data_Q)
    # END main logic
    time_stop = monotonic()
    

    df = pd.concat({'𝜇(p)': data_P, '𝜇(q)': data_Q, 'Δ': delta}).unstack().unstack(0)
    
    result[norm_name] = {
        'data': df,
        'time': time_stop - time_start
    }

    
def title(message):
    print('\n', message.upper(), '\n', '-'*len(message), '\n', sep='')
    
    
title('Calculation time')
for norm_name in norm_collection:
    print('%10s : %.4f seconds' % (norm_name, result[norm_name]['time']))

title('Results')
idx = range(0,len(P),3)
for norm_name in norm_collection:
    print(f'{norm_name}:')
    display(result[norm_name]['data'].iloc[idx].round(4)
        .style.background_gradient(cmap='Reds', subset=[(n,'Δ') for n in N], vmax=8))


CALCULATION TIME
----------------

     basic : 0.2401 seconds
    linalg : 0.2607 seconds
     power : 0.2005 seconds

RESULTS
-------

basic:


linalg:


power:
